## Start on frontend Here!
Setup environment first 
import those dependecies


In [83]:
import os
from dotenv import load_dotenv
from openai import OpenAI
#if imports are not working try using line below  to update terminal to grab dependices from environment.yml 
# conda env update --file environment.yml --prune 

load_dotenv(override=True)  # take environment variables from env.

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)




# create response bot

In [2]:
response = client.chat.completions.create(
    model="gpt-35-turbo-16k", #correct gpt-ai
    messages=[
        {"role": "system", "content": "you are an expert notetaker who has 10 years of experience of writing notes, Summarzing big paragraphs, and stories. Your task is to summarize the paragraphs from users input"},
        {"role": "user", "content": "**Title: The Fallout of Betrayal**  The arena was electric, the crowd buzzing with anticipation as the lights dimmed and the familiar notes of Cody Rhodes' entrance music echoed throughout the venue.The Undisputed WWE Champion, a symbol of resilience and determination, made his way to the ring, the championship belt glistening on his shoulder.But tonight, the air was charged with tension; his first appearance since the shocking betrayal at WWE Elimination Chamber loomed heavy over him. As he climbed the steps, the audience erupted in a mix of cheers and murmurs, all eyes drawn to the bruised and battered visage of their champion.Cody's left eye was swollen, a stark reminder of the chaos that had unfolded just days prior when John Cena, a man he once idolized, turned heel and attacked him.The sight of his father’s legacy being tarnished by a betrayal he never saw coming weighed heavily on him. Cody grabbed the microphone, his voice steady but laced with emotion.He began by reflecting on the moment his daughter had innocently asked him about the injury.“I told her I ran into a wall,” he confessed, allowing a brief, bittersweet smile to cross his lips.“But that wasn’t entirely a lie.I ran into a wall built by The Rock, Travis Scott, and John Cena.”  There was a palpable shift in the atmosphere as Cody continued, reminiscing about the days when he and Cena shared the road, a time when Cena was nothing short of a hero in his eyes.“I used to look up to him,” he admitted, his voice thick with nostalgia.“But now, I find myself standing here, asking why he did it.What drove him to betray me? To turn his back on everything we built together?”  The crowd began to respond, some chanting Cena's name while others shouted support for Cody.He took a moment to absorb it all, the duality of admiration and betrayal swirling around him.“John will likely justify his actions,” Cody pressed on, “but there’s no justification for what he did.A great man gave up, and it’s a tragedy.”  As he spoke, Cody shifted his focus to the support of the fans who had rallied behind him during his title run at WrestleMania 40.He recalled the moment of victory, the elation that came with the championship, and the bittersweet feeling of sharing it with a man who now stood as an adversary.“John celebrated with me that night,” he reminded the audience, “but he also told me that the title gets heavier every day.I guess he wasn’t strong enough to carry it anymore.”  With fire igniting in his eyes, Cody concluded his message.“John Cena may be the last of his kind, but I am not.I’m the American Nightmare, and if he wants a piece of me, he knows where to find me.I’ll be waiting.”  The crowd erupted, a mixture of cheers and applause, as Cody dropped the microphone and stood tall, the champion in the face of adversity.He had laid bare his heart, revealing his vulnerability and strength in one powerful moment.The road ahead was uncertain, but Cody Rhodes was ready to face whatever came next—because in the world of wrestling, the only certainty was the fight.And he was prepared to fight for the truth, for his legacy, and for the future he envisioned for his daughter.The walls may have been built around him, but he was determined to break them down, one match at a time."}
    ],
    temperature=0.7
)


system_output = response.choices[0].message.content
print(system_output)


In this electrifying WWE arena, Cody Rhodes, the Undisputed WWE Champion, made his first appearance since the shocking betrayal at WWE Elimination Chamber. The crowd was buzzing with anticipation as Cody entered the ring, his bruised and battered face a stark reminder of the betrayal by John Cena. Reflecting on the moment his daughter asked about his injury, Cody confessed that he had run into a wall built by The Rock, Travis Scott, and John Cena. He expressed his disappointment in Cena, someone he once idolized, and questioned why he had turned his back on their shared history. Despite the betrayal, Cody acknowledged the support of his fans and the bittersweet feeling of sharing his championship victory with an adversary. He declared himself the American Nightmare and vowed to be ready for any challenge that came his way. Cody was determined to fight for the truth, his legacy, and the future he envisioned for his daughter, breaking down the walls that surrounded him with every match.



Cross-validation scores: [0.95194979 0.94474013 0.94703026 0.94883267 0.95175894]
Mean CV score: 0.949 (+/- 0.006)

Cross-validation scores: [0.95194979 0.94474013 0.94703026 0.94883267 0.95175894]
Mean CV score: 0.949 (+/- 0.006)

Overall accuracy: 0.9388536652600776

Feature Importance Ranking:
                       Feature  Importance
11  NoOfOtherSpecialCharsInURL    0.414180
6             LetterRatioInURL    0.135448
3                    TLDLength    0.122200
4                NoOfSubDomain    0.121101
7              DegitRatioInURL    0.098863
0                    URLLength    0.067547
2         CharContinuationRate    0.032729
1                 DomainLength    0.006465
5             NoOfLettersInURL    0.001467
8              NoOfEqualsInURL    0.000000
9               NoOfQMarkInURL    0.000000
10          NoOfAmpersandInURL    0.000000

Model trained and saved successfully!


## WORKING MODEL OUTPUTS 94% CROSS VALIDATION
## 93% OVERALL FOR OUR WHOLE DATASET
###                                BEFORE OUR IMPLEMENTATION OF OPENAI 
###                                XGBOOST ALONE




In [85]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
import joblib
import numpy as np
import unittest
import os
import sys
from sklearn.exceptions import NotFittedError


###setup environment###

load_dotenv(override=True)  # take environment variables from env.

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)
# Specify the path to your CSV file
csv_file_path = '/Users/jorgegonzalezjr/seniorproject_llm/data/Phishing_url_or_not.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)


parsed_data = ['URLLength','DomainLength','CharContinuationRate','TLDLength','NoOfSubDomain',
            'NoOfLettersInURL',
            'LetterRatioInURL',
            'DegitRatioInURL',
            'NoOfEqualsInURL',
            'NoOfQMarkInURL',
            'NoOfAmpersandInURL',
            'NoOfOtherSpecialCharsInURL']

# # Prepare X and y
x = df[parsed_data]
y = df['label'] 

# Initialize model with best parameters
model = xgb.XGBClassifier(
    alpha=0.1,          # Best alpha from grid search
    gamma=0.1,          # Best gamma from grid search
    reg_lambda=0.1,     # Best lambda from grid search
    max_depth=3,        # Changed back to 3
    random_state=42,
    learning_rate=0.3,
    n_estimators= 23
)

# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, x, y, cv=kfold, scoring='accuracy')

# Print cross-validation results
print("\nCross-validation scores:", cv_scores)
print("Mean CV score: {:.3f} (+/- {:.3f})".format(cv_scores.mean(), cv_scores.std() * 2))

# Train the final model on all data
model.fit(x, y)


# Print cross-validation results
print("\nCross-validation scores:", cv_scores)
print("Mean CV score: {:.3f} (+/- {:.3f})".format(cv_scores.mean(), cv_scores.std() * 2))

# Train the final model on all data
model.fit(x, y)

# Get overall accuracy
accuracy = model.score(x, y)
print("\nOverall accuracy:", accuracy)

# Feature importance analysis
feature_importance = pd.DataFrame({
    'Feature': parsed_data,
    'Importance': model.feature_importances_
})

# Sort and print feature importance
print("\nFeature Importance Ranking:")
print(feature_importance.sort_values('Importance', ascending=False))

# Create the 'saved_models' directory if it doesn't exist
os.makedirs('saved_models', exist_ok=True)

# Now you can save the model
joblib.dump(model, 'saved_models/xgb_model_tuned.joblib')

print("\nModel trained and saved successfully!")


##keep in mind this is the code for the estimators





Cross-validation scores: [0.95194979 0.94474013 0.94703026 0.94883267 0.95175894]
Mean CV score: 0.949 (+/- 0.006)

Cross-validation scores: [0.95194979 0.94474013 0.94703026 0.94883267 0.95175894]
Mean CV score: 0.949 (+/- 0.006)

Overall accuracy: 0.9388536652600776

Feature Importance Ranking:
                       Feature  Importance
11  NoOfOtherSpecialCharsInURL    0.414180
6             LetterRatioInURL    0.135448
3                    TLDLength    0.122200
4                NoOfSubDomain    0.121101
7              DegitRatioInURL    0.098863
0                    URLLength    0.067547
2         CharContinuationRate    0.032729
1                 DomainLength    0.006465
5             NoOfLettersInURL    0.001467
8              NoOfEqualsInURL    0.000000
9               NoOfQMarkInURL    0.000000
10          NoOfAmpersandInURL    0.000000

Model trained and saved successfully!


# NOW LETS IMPLEMENT OPENAI PROMPTS BABY
## ----------------------- 1. XGBoost 2. LLM Prompt Model -----------------------

            
            1. First step is to set up our openai environment.
            2. implement xgboost
            3. 

In [86]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
import joblib
import numpy as np
import unittest
import os
import sys
from sklearn.exceptions import NotFittedError

##################### 1. First step is to set up our openai environment. #####################
load_dotenv(override=True)  # take environment variables from env.

client = OpenAI(
    base_url=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY")
)

##################### 1. First step is to set up our openai environment. #####################



##################### 2. implement xgboost##########################################
# Specify the path to your CSV file
csv_file_path = '/Users/jorgegonzalezjr/seniorproject_llm/data/Phishing_url_or_not.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)


parsed_data = ['URLLength','DomainLength','CharContinuationRate','TLDLength','NoOfSubDomain',
            'NoOfLettersInURL',
            'LetterRatioInURL',
            'DegitRatioInURL',
            'NoOfEqualsInURL',
            'NoOfQMarkInURL',
            'NoOfAmpersandInURL',
            'NoOfOtherSpecialCharsInURL']

# Prepare X and y
x = df[parsed_data]  # Features
y = df['label']      # Target (Spam or Not Spam)

# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Initialize XGBoost model
model = xgb.XGBClassifier(
    alpha=0.1,  
    gamma=0.1,  
    reg_lambda=0.1,  
    max_depth=3,  
    random_state=42,  
    learning_rate=0.3,  
    n_estimators=19
)

# Train the model
model.fit(X_train, y_train)

# Get predictions on test set
prediction = model.predict(X_test)
print("XGBoost Prediction:", prediction)

# Evaluate accuracy
accuracy = accuracy_score(y_test, prediction)
print(f"Accuracy: {accuracy * 100:.2f}%")
# Perform k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(model, x, y, cv=kfold, scoring='accuracy')

# Print cross-validation results
print("\nCross-validation scores:", cv_scores)
print("Mean CV score: {:.3f} (+/- {:.3f})".format(cv_scores.mean(), cv_scores.std() * 2))


# Train the final model on all data
model.fit(x, y)

# Get overall accuracy
accuracy = model.score(x, y)
print("\nOverall accuracy:", accuracy)


# Create the 'saved_models' directory if it doesn't exist
os.makedirs('saved_models', exist_ok=True)

# Now you can save the model
joblib.dump(model, 'saved_models/xgb_model_tuned.joblib')

print("\nModel trained and saved successfully!")
print("\nNow sending uncertain cases to LLM to guarantee accuracy...")
##################### 2. implement xgboost##########################################


################################### 3. LLM PROMPT CODE ###########################################



XGBoost Prediction: [1 0 0 ... 1 0 1]
Accuracy: 93.91%

Cross-validation scores: [0.93909964 0.92983312 0.93560084 0.94124133 0.94223796]
Mean CV score: 0.938 (+/- 0.009)

Overall accuracy: 0.9315676753111813

Model trained and saved successfully!

Now sending uncertain cases to LLM to guarantee accuracy...
